# Part 3 - Stage 2 Data Manipulation

The purpose of this library is to manipulate stage 2 data into useable training data. 

The inputs you need for this are:
/data/MNCAATourneySeeds.csv
/data/MRegularSeasonDetailedResults.csv

## Library Imports

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pkg_resources

from binaryTree import Node
from PIL import Image, ImageDraw

from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [2]:
cwd = os.getcwd()

## Data Manipulation

### Regular Season Data Analysis

In [10]:
# Existing code from data manipulation section. Only run if needed. 
cwd = os.getcwd()

# tourney_cresults = pd.read_csv(cwd + '/data/MNCAATourneyCompactResults.csv')
# seeds = pd.read_csv(cwd + '/data/MNCAATourneySeeds.csv')
seeds = pd.read_csv(cwd + '/data_stage2/MNCAATourneySeeds.csv')
season_dresults = pd.read_csv(cwd +'/data_stage2/MRegularSeasonDetailedResults.csv')

targetYear = 2003
# tourney_cresults = tourney_cresults.loc[tourney_cresults['Season'] >= targetYear]

training_set = pd.read_csv("training_set.csv")
record = pd.read_csv('record.csv')

seeds['Seed'] =  pd.to_numeric(seeds['Seed'].str[1:3], downcast='integer',errors='coerce')

def delta_seed(row):
    cond = (seeds['Season'] == row['Season'])
    return seeds[cond & (seeds['TeamID'] == row['Team1'])]['Seed'].iloc[0] - seeds[cond & (seeds['TeamID'] == row['Team2'])]['Seed'].iloc[0]

# function to, given a row, calculate what the difference between the two seeds was. 
#Function to look up 
def delta_winPct(row):
    cond1 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team1'])
    cond2 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team2'])
    return (record[cond1]['wins']/record[cond1]['games']).mean() - (record[cond2]['wins']/record[cond2]['games']).mean()

def get_points_against(row):
    wcond = (dfW['Season'] == row['Season']) & (dfW['WTeamID'] == row['WTeamID']) 
    fld1 = 'LScore'
    lcond = (dfL['Season'] == row['Season']) & (dfL['LTeamID'] == row['WTeamID']) 
    fld2 = 'WScore'
    retVal = dfW[wcond][fld1].sum()
    if len(dfL[lcond][fld2]) > 0:
        retVal = retVal + dfL[lcond][fld2].sum() 
    return retVal

def get_points_for(row):
    wcond = (dfW['Season'] == row['Season']) & (dfW['WTeamID'] == row['WTeamID']) 
    fld1 = 'WScore'
    lcond = (dfL['Season'] == row['Season']) & (dfL['LTeamID'] == row['WTeamID']) 
    fld2 = 'LScore'
    retVal = dfW[wcond][fld1].sum()
    if len(dfL[lcond][fld2]) > 0:
        retVal = retVal + dfL[lcond][fld2].sum() 
    return retVal

def get_remaining_stats(row, field):
    wcond = (dfW['Season'] == row['Season']) & (dfW['WTeamID'] == row['WTeamID']) 
    fld1 = 'W' + field
    lcond = (dfL['Season'] == row['Season']) & (dfL['LTeamID'] == row['WTeamID']) 
    fld2 = 'L'+ field
    retVal = dfW[wcond][fld1].sum()
    if len(dfL[lcond][fld2]) > 0:
        retVal = retVal + dfL[lcond][fld2].sum()
    return retVal

def delta_stat(row, field):
    cond1 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team1'])
    cond2 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team2'])
    return (record[cond1][field]/record[cond1]['games']).mean() - (record[cond2][field]/record[cond2]['games']).mean()

Ok, so now we have a trained model. Next we need to find sumission data.

The kaggle competition provides a sample submission.csv file that contains a matchup ID, and a default prediction value. 

In [11]:
sub = pd.read_csv(cwd + '/data_stage2/MSampleSubmissionStage2.csv')
sub

,ID,Pred
0,2021_1101_1104,0.5
1,2021_1101_1111,0.5
2,2021_1101_1116,0.5
3,2021_1101_1124,0.5
4,2021_1101_1140,0.5
...,...,...
2273,2021_1452_1457,0.5
2274,2021_1452_1458,0.5
2275,2021_1455_1457,0.5
2276,2021_1455_1458,0.5


Split this string into Team IDs and year

In [12]:
sub['Season'], sub['Team1'], sub['Team2'] = sub['ID'].str.split('_').str
sub[['Season', 'Team1', 'Team2']] = sub[['Season', 'Team1', 'Team2']].apply(pd.to_numeric)
sub

,ID,Pred,Season,Team1,Team2
0,2021_1101_1104,0.5,2021,1101,1104
1,2021_1101_1111,0.5,2021,1101,1111
2,2021_1101_1116,0.5,2021,1101,1116
3,2021_1101_1124,0.5,2021,1101,1124
4,2021_1101_1140,0.5,2021,1101,1140
...,...,...,...,...,...
2273,2021_1452_1457,0.5,2021,1452,1457
2274,2021_1452_1458,0.5,2021,1452,1458
2275,2021_1455_1457,0.5,2021,1455,1457
2276,2021_1455_1458,0.5,2021,1455,1458


Calculate the deltaSeed and deltaWinPct features

In [13]:
sub['deltaSeed'] = sub.apply(delta_seed,axis=1)
# sub['deltaMO'] = sub.apply(delta_ord,axis=1)
sub['deltaWinPct'] = sub.apply(delta_winPct,axis=1)
sub

,ID,Pred,Season,Team1,Team2,deltaSeed,deltaWinPct
0,2021_1101_1104,0.5,2021,1101,1104,12,0.026087
1,2021_1101_1111,0.5,2021,1101,1111,-2,0.284420
2,2021_1101_1116,0.5,2021,1101,1116,11,0.040373
3,2021_1101_1124,0.5,2021,1101,1124,13,-0.090580
4,2021_1101_1140,0.5,2021,1101,1140,8,0.066087
...,...,...,...,...,...,...,...
2273,2021_1452_1457,0.5,2021,1452,1457,-9,-0.291667
2274,2021_1452_1458,0.5,2021,1452,1458,-6,0.080460
2275,2021_1455_1457,0.5,2021,1455,1457,-1,-0.221491
2276,2021_1455_1458,0.5,2021,1455,1458,2,0.150635


Now, caluclate the rest of our stats. This will take a while.

In [14]:
# cut to slides
rawCols = ['PointsFor','PointsAgainst','FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF']

for rawCol in rawCols:
    print("Processing",rawCol)
    sub['delta' + rawCol] = sub.apply(delta_stat,args=(rawCol,),axis=1)

Processing PointsFor
Processing PointsAgainst
Processing FGM
Processing FGA
Processing FGM3
Processing FGA3
Processing FTM
Processing FTA
Processing OR
Processing DR
Processing Ast
Processing TO
Processing Stl
Processing Blk
Processing PF


In [15]:
sub.to_csv("training_set_stage2.csv", index=False)
sub

,ID,Pred,Season,Team1,Team2,deltaSeed,deltaWinPct,deltaPointsFor,deltaPointsAgainst,deltaFGM,...,deltaFGA3,deltaFTM,deltaFTA,deltaOR,deltaDR,deltaAst,deltaTO,deltaStl,deltaBlk,deltaPF
0,2021_1101_1104,0.5,2021,1101,1104,12,0.026087,-3.262319,-8.027536,-0.626087,...,-8.834783,0.526087,1.620290,-0.791304,-3.140580,4.040580,-0.398551,0.714493,-1.333333,0.805797
1,2021_1101_1111,0.5,2021,1101,1111,-2,0.284420,8.137681,-6.510870,3.965580,...,-3.601449,0.284420,1.170290,0.733696,0.784420,6.840580,2.309783,1.806159,-0.166667,-2.344203
2,2021_1101_1116,0.5,2021,1101,1116,11,0.040373,-6.088509,-8.939441,-2.290373,...,-1.613354,-1.781056,-1.091615,-0.962733,-3.316770,3.245342,0.613354,1.312112,-2.142857,1.703416
3,2021_1101_1124,0.5,2021,1101,1124,13,-0.090580,-8.070652,-4.677536,-3.909420,...,-3.143116,1.951087,3.045290,-1.724638,0.451087,1.132246,1.268116,0.389493,-0.750000,0.489130
4,2021_1101_1140,0.5,2021,1101,1140,8,0.066087,-1.255652,-7.100870,-1.226087,...,-1.514783,1.666087,3.566957,1.288696,-5.453913,2.013913,0.434783,4.667826,0.160000,1.139130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2021_1452_1457,0.5,2021,1452,1457,-9,-0.291667,-2.245370,5.319444,-1.750000,...,-3.625000,2.453704,2.472222,0.680556,-3.458333,-1.263889,-2.037037,-0.268519,0.476852,0.722222
2274,2021_1452_1458,0.5,2021,1452,1458,-6,0.080460,7.675607,7.800766,1.816092,...,-3.425287,5.335888,8.532567,5.199234,-1.034483,0.467433,3.043423,1.619413,-0.941252,0.049808
2275,2021_1455_1457,0.5,2021,1455,1457,-1,-0.221491,-7.646930,2.471491,-4.188596,...,0.304825,0.820175,0.776316,-2.208333,-2.984649,-2.199561,-3.087719,-2.013158,1.046053,0.078947
2276,2021_1455_1458,0.5,2021,1455,1458,2,0.150635,2.274047,4.952813,-0.622505,...,0.504537,3.702359,6.836661,2.310345,-0.560799,-0.468240,1.992740,-0.125227,-0.372051,-0.593466


In [16]:
sub

,ID,Pred,Season,Team1,Team2,deltaSeed,deltaWinPct,deltaPointsFor,deltaPointsAgainst,deltaFGM,...,deltaFGA3,deltaFTM,deltaFTA,deltaOR,deltaDR,deltaAst,deltaTO,deltaStl,deltaBlk,deltaPF
0,2021_1101_1104,0.5,2021,1101,1104,12,0.026087,-3.262319,-8.027536,-0.626087,...,-8.834783,0.526087,1.620290,-0.791304,-3.140580,4.040580,-0.398551,0.714493,-1.333333,0.805797
1,2021_1101_1111,0.5,2021,1101,1111,-2,0.284420,8.137681,-6.510870,3.965580,...,-3.601449,0.284420,1.170290,0.733696,0.784420,6.840580,2.309783,1.806159,-0.166667,-2.344203
2,2021_1101_1116,0.5,2021,1101,1116,11,0.040373,-6.088509,-8.939441,-2.290373,...,-1.613354,-1.781056,-1.091615,-0.962733,-3.316770,3.245342,0.613354,1.312112,-2.142857,1.703416
3,2021_1101_1124,0.5,2021,1101,1124,13,-0.090580,-8.070652,-4.677536,-3.909420,...,-3.143116,1.951087,3.045290,-1.724638,0.451087,1.132246,1.268116,0.389493,-0.750000,0.489130
4,2021_1101_1140,0.5,2021,1101,1140,8,0.066087,-1.255652,-7.100870,-1.226087,...,-1.514783,1.666087,3.566957,1.288696,-5.453913,2.013913,0.434783,4.667826,0.160000,1.139130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2021_1452_1457,0.5,2021,1452,1457,-9,-0.291667,-2.245370,5.319444,-1.750000,...,-3.625000,2.453704,2.472222,0.680556,-3.458333,-1.263889,-2.037037,-0.268519,0.476852,0.722222
2274,2021_1452_1458,0.5,2021,1452,1458,-6,0.080460,7.675607,7.800766,1.816092,...,-3.425287,5.335888,8.532567,5.199234,-1.034483,0.467433,3.043423,1.619413,-0.941252,0.049808
2275,2021_1455_1457,0.5,2021,1455,1457,-1,-0.221491,-7.646930,2.471491,-4.188596,...,0.304825,0.820175,0.776316,-2.208333,-2.984649,-2.199561,-3.087719,-2.013158,1.046053,0.078947
2276,2021_1455_1458,0.5,2021,1455,1458,2,0.150635,2.274047,4.952813,-0.622505,...,0.504537,3.702359,6.836661,2.310345,-0.560799,-0.468240,1.992740,-0.125227,-0.372051,-0.593466
